In [22]:
import os
from pathlib import Path
import shutil
import subprocess
import re

### Pastas relevantes

In [26]:
python_input_dir = Path("QuixBugs/python_programs")
python_output_dir = Path("QuixBugs/python_programs_corrected_by_ai")
python_test_dir = Path("QuixBugs/python_testcases")
java_input_dir = Path("QuixBugs/java_programs")
java_output_dir = Path("QuixBugs/java_programs_corrected_by_ai")
java_test_dir = Path("QuixBugs/java_testcases/junit")

junit_jar_path = "junit-4.13.2.jar"

python_output_dir.mkdir(exist_ok=True)
java_output_dir.mkdir(exist_ok=True)

### Chamada da IA para correção dos bugs

In [ ]:
def correct_with_ai(code: str, language: str, ai: str):
    print(code)

In [ ]:
def process_files(input_dir: Path, output_dir: Path, language: str, ai: str):
    for file_path in input_dir.glob("*.java" if language == "java" else "*.py"):
        with open(file_path, "r", encoding="utf-8") as f:
            code = f.read()
        
        corrected_code = correct_with_ai(code, language, ai)
        
        output_file = output_dir / file_path.name
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(corrected_code)
        print(f"[{language.upper()}] Corrigido e salvo: {output_file}")

In [ ]:
process_files(python_input_dir, python_output_dir, "python", "")

In [ ]:
process_files(java_input_dir, java_output_dir, "java", "")

### Pré-processamento dos arquivos de teste com as classes corretas

In [ ]:
for test_file in python_test_dir.glob("test_*.py"):
    with open(test_file, "r", encoding="utf-8") as file:
        content = file.read()

    if "from python_programs." in content:
        new_content = content.replace(
            "from python_programs.", "from python_programs_corrected_by_ai."
        )

        with open(test_file, "w", encoding="utf-8") as file:
            file.write(new_content)
            
        # print(f"Modificado: {test_file.name}")
    else:
        print(f"Nenhuma modificação: {test_file.name}")

In [ ]:
for test_file in java_test_dir.glob("*_TEST.java"):
    with open(test_file, "r", encoding="utf-8") as file:
        content = file.read()

    if "java_programs" in content:
        new_content = content.replace(
            "java_programs", "java_programs_corrected_by_ai"
        )

        with open(test_file, "w", encoding="utf-8") as file:
            file.write(new_content)

    #     print(f"Modificado: {test_file.name}")
    else:
        print(f"Nenhuma modificação: {test_file.name}")

### Testes de python

In [ ]:
python_test_dir = Path("QuixBugs/python_testcases")
python_test_files = list(python_test_dir.glob("test_*.py"))
print(f"Executando {len(python_test_files)} arquivos de teste...\n")

for test_file in python_test_files:
    print(f"==> Rodando testes em: {test_file.name}")
    
    result = subprocess.run(
        ["pytest", str(test_file), "--tb=no", "--maxfail=1000", "-q"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    summary_line = next((line for line in result.stdout.splitlines() if re.search(r"(failed|passed)", line)), "")

    match_fail = re.search(r"(\d+)\s+failed", summary_line)
    match_total = re.findall(r"(\d+)\s+(?:passed|failed)", summary_line)

    failed_count = int(match_fail.group(1)) if match_fail else 0
    total_count = sum(int(x) for x in match_total)

    print(f"{test_file.name}: {failed_count} falha(s) de {total_count}\n")

### Compilação dos arquivos de java

In [ ]:
java_files = [str(file) for file in java_input_dir.glob("*.java")]

if java_files:
    compile_command = [
        "javac",
        *java_files 
    ]
    
    result = subprocess.run(compile_command, capture_output=True, text=True)

    if result.returncode != 0:
        print("Erro na compilação:")
        print(result.stderr)
    else:
        print("Compilação bem-sucedida.")
else:
    print("Nenhum arquivo .java encontrado no diretório especificado.")


Compilação bem-sucedida.


In [28]:
java_test_files = list(java_test_dir.glob("*.java"))

if not java_test_files:
    print("Nenhum arquivo .java encontrado.")
else:
    compile_command = [
        "javac",
        "-cp", f".:{junit_jar_path}:{java_input_dir}",
    ] + [str(file) for file in java_test_files]  

    result = subprocess.run(compile_command, capture_output=True, text=True)

    if result.returncode != 0:
        print("Erro na compilação:")
        print(result.stderr)
    else:
        print("Compilação bem-sucedida.")


Erro na compilação:
QuixBugs/java_testcases/junit/MINIMUM_SPANNING_TREE_TEST.java:13: error: package java_programs does not exist
import java_programs.MINIMUM_SPANNING_TREE;
                    ^
QuixBugs/java_testcases/junit/MINIMUM_SPANNING_TREE_TEST.java:14: error: package java_programs does not exist
import java_programs.Node;
                    ^
QuixBugs/java_testcases/junit/MINIMUM_SPANNING_TREE_TEST.java:15: error: package java_programs does not exist
import java_programs.WeightedEdge;
                    ^
QuixBugs/java_testcases/junit/MINIMUM_SPANNING_TREE_TEST.java:20: error: cannot find symbol
    Node node1 = new Node("1");
    ^
  symbol:   class Node
  location: class MINIMUM_SPANNING_TREE_TEST
QuixBugs/java_testcases/junit/MINIMUM_SPANNING_TREE_TEST.java:21: error: cannot find symbol
    Node node2 = new Node("2");
    ^
  symbol:   class Node
  location: class MINIMUM_SPANNING_TREE_TEST
QuixBugs/java_testcases/junit/MINIMUM_SPANNING_TREE_TEST.java:22: error: cannot fi

In [30]:
java_test_dir = Path("QuixBugs/java_testcases/junit")
java_test_files = list(java_test_dir.glob("*_TEST.class"))  # Usando arquivos .class já compilados
print(f"Executando {len(java_test_files)} arquivos de teste...\n")

for test_file in java_test_files:
    print(f"==> Rodando testes em: {test_file.name}")

    test_package_dir = test_file.parent

    class_name = test_file.stem

    test_command = [
        "java",
        "-cp", f".:{junit_jar_path}:{java_test_dir}",
        "org.junit.runner.JUnitCore",
        f"{test_package_dir.stem}.{class_name}"
    ]

    result = subprocess.run(test_command, capture_output=True, text=True)

    summary_line = next((line for line in result.stdout.splitlines() if re.search(r"(failed|passed)", line)), "")

    match_fail = re.search(r"(\d+)\s+failed", summary_line)
    match_total = re.findall(r"(\d+)\s+(?:passed|failed)", summary_line)

    failed_count = int(match_fail.group(1)) if match_fail else 0
    total_count = sum(int(x) for x in match_total)

    print(f"{test_file.name}: {failed_count} falha(s) de {total_count}\n")


Executando 0 arquivos de teste...

